# Quantitative Momentum Strategy

In this exercise, we will use the principles of momentum investing as it is applied to stocks and replicate that for cryptocurrencies. Momentum investing means investing in assets that have increased in price the most. 

We will build an investing strategy that selects the top 20 cryptocurrencies with the highest price momentum from the list of top 200 cryptocurrencies. We will then calculate recommended trades for an equal weight portfolio of these 20 cryptocurrencies. 

# Library Imports 

The first thing we need to do is import the open-source software libraries that we'll be using in this exercise. 

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

# Importing List of Cryptocurrencies

The next thing we need to do is import the list of top 200 cryptocurrencies as on https://www.coingecko.com/.

The cryptocurrencies in this list may change over time so the ideal way to get the list of top 200 cryptocurrencies is to connect to the Coingecko API and get the real time data from their API database. The API documentation can be found here: https://www.coingecko.com/en/api/documentation.

Since we will be using the free API version, there is no need to get an API key. However, if we choose a different plan, an API key will be required to access the data from their API database. We will be using the /coins/markets API end point to get the list of top 200 cryptocurrencies, their ticker, price, market capitalization and other relevant data.

In [2]:
api_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=200&page=1&sparkline=false&price_change_percentage='24h%2C7d%2C14d%2C30d%2C200d%2C1y%2C'"
data = requests.get(api_url).json()

# Creating a Pandas DataFrame and parsing our API call

Next, we will parse the data that we obtained above and store it in a pandas dataframe. Initially, we will include the following columns in our dataframe - Name, Ticker, Current Price, Market Capitalization, 200D Price Return, and Number of Tokens to Buy.

In [3]:
my_columns = ['Name', 'Ticker', 'Current Price', 'Market Capitalization', '200D Price Return', 'Number of Tokens to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Name,Ticker,Current Price,Market Capitalization,200D Price Return,Number of Tokens to Buy


In [4]:
for i in data:
    name = i['name']
    ticker = i['symbol']
    price = i['current_price']
    market_cap = i['market_cap']
    price_change_percentage_200d = i['price_change_percentage_200d_in_currency']
    final_dataframe = final_dataframe.append(
    pd.Series(
        [
            name,
            ticker,
            price,
            market_cap,
            price_change_percentage_200d,
            'N/A',
        ],
            index = my_columns,
    ),
        ignore_index = True
)
final_dataframe

,Name,Ticker,Current Price,Market Capitalization,200D Price Return,Number of Tokens to Buy
0,Bitcoin,btc,39142,742988405806,-20.009486,N/A
1,Ethereum,eth,2596.04,311193529447,-19.477657,N/A
2,Tether,usdt,1.001,80181494275,0.04619,N/A
3,BNB,bnb,377.37,63436097483,-16.16903,N/A
4,USD Coin,usdc,1.0,52310423078,0.189143,N/A
...,...,...,...,...,...,...
195,Gemini Dollar,gusd,1.0,286012046,0.829234,N/A
196,Ronin,ron,1.92,284211846,None,N/A
197,WazirX,wrx,0.61647,281461624,-58.867196,N/A
198,SafeMoon [OLD],safemoon,0.0,277800683,-75.570325,N/A


# Removing Low-Momentum Cryptocurrencies

Since we are building an investment strategy that will select the top 20 highest momentum cryptocurrencies from the list of top 200, we will therefore remove cryptocurrencies that fall below this momentum threshold. We will then sort the pandas dataframe by cryptocurrrencies' 200D price return and drop all cryptocurrencies outside the top 20. 

In [5]:
final_dataframe.sort_values('200D Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:20]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Name,Ticker,Current Price,Market Capitalization,200D Price Return,Number of Tokens to Buy
0,156,Metis Token,metis,146.63,435516616,1920.284828,N/A
1,165,Dogelon Mars,elon,0.000001,392319341,1006.400301,N/A
2,108,Bitkub Coin,kub,8.05,712594961,958.10944,N/A
3,65,Gala,gala,0.216976,1637095180,947.836135,N/A
4,87,Kadena,kda,6.26,1072154967,701.520039,N/A
5,189,Flux,flux,1.33,303972544,651.489546,N/A
6,38,The Sandbox,sand,2.81,3162051318,322.734858,N/A
7,141,Velas,vlx,0.221629,503384006,318.62598,N/A
8,90,Frax Share,fxs,17.34,1000965750,317.280256,N/A
9,6,Terra,luna,95.86,34885934028,247.418194,N/A


# Calculating the number of Tokens to Buy

Next, we will calculate the number of tokens we need to buy for an equal-weight investment distribution. 

In [6]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [7]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Tokens to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Current Price'])
final_dataframe

,index,Name,Ticker,Current Price,Market Capitalization,200D Price Return,Number of Tokens to Buy
0,156,Metis Token,metis,146.63,435516616,1920.284828,340
1,165,Dogelon Mars,elon,0.000001,392319341,1006.400301,70265279536
2,108,Bitkub Coin,kub,8.05,712594961,958.10944,6211
3,65,Gala,gala,0.216976,1637095180,947.836135,230440
4,87,Kadena,kda,6.26,1072154967,701.520039,7987
5,189,Flux,flux,1.33,303972544,651.489546,37593
6,38,The Sandbox,sand,2.81,3162051318,322.734858,17793
7,141,Velas,vlx,0.221629,503384006,318.62598,225602
8,90,Frax Share,fxs,17.34,1000965750,317.280256,2883
9,6,Terra,luna,95.86,34885934028,247.418194,521


# Building a Better (more realistic) Momentum Strategy

Next, we want to differentiate between "low quality" and "high quality" momentum cryptocurrencies. High quality momentum cryptocurrencies show slow and steady performance over long periods of time. Low quality momentum cryptocurrencies might not show any momentum for a long time, and then surge upwards. 

The reason why we should pick high quality momentum cryptocurrencies is because low quality momentum can often be caused by pump and dump or short-term news that is unlikely to be repeated in the future (such as partnerships, funding, etc).

To identify high quality momentum, we're going to build a strategy that selects cryptocurrencies from the highest percentiles of: 24h price returns, 7d price returns, 14d price returns, 30d price returns, 200d price returns, and 1 year price returns.

In [8]:
hqm_columns = [
    'Name',
    'Ticker', 
    'Current Price',
    'HQM Score',
    'Market Capitalization', 
    '24H Price Return',
    '24H Return Percentile',
    '7D Price Return',
    '7D Return Percentile',
    '14D Price Return',
    '14D Return Percentile',
    '30D Price Return',
    '30D Return Percentile',
    '200D Price Return',
    '200D Return Percentile',
    '1Y Price Return',
    '1Y Return Percentile',
    'Number of Tokens to Buy',
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)
hqm_dataframe

,Name,Ticker,Current Price,HQM Score,Market Capitalization,24H Price Return,24H Return Percentile,7D Price Return,7D Return Percentile,14D Price Return,14D Return Percentile,30D Price Return,30D Return Percentile,200D Price Return,200D Return Percentile,1Y Price Return,1Y Return Percentile,Number of Tokens to Buy


In [9]:
for i in data:
    name = i['name']
    ticker = i['symbol']
    price = i['current_price']
    hqm_score = 'N/A'
    market_cap = i['market_cap']
    price_change_percentage_24h = i['price_change_24h']
    percentile_return_24h = 'N/A'
    price_change_percentage_7d = i['price_change_percentage_7d_in_currency']
    percentile_return_7d = 'N/A'
    price_change_percentage_14d = i['price_change_percentage_14d_in_currency']
    percentile_return_14d = 'N/A'
    price_change_percentage_30d = i['price_change_percentage_30d_in_currency']
    percentile_return_30d = 'N/A'
    price_change_percentage_200d = i['price_change_percentage_200d_in_currency']
    percentile_return_200d = 'N/A'
    price_change_percentage_1y = i['price_change_percentage_1y_in_currency']
    percentile_return_1y = 'N/A'

    hqm_dataframe = hqm_dataframe.append(
    pd.Series(
        [
            name,
            ticker,
            price,
            hqm_score,
            market_cap,
            price_change_percentage_24h,
            percentile_return_24h,
            price_change_percentage_7d,
            percentile_return_7d,
            price_change_percentage_14d,
            percentile_return_14d,
            price_change_percentage_30d,
            percentile_return_30d,
            price_change_percentage_200d,
            percentile_return_200d,
            price_change_percentage_1y,
            percentile_return_1y,
            'N/A',
        ],
            index = hqm_columns,
    ),
        ignore_index = True
)
hqm_dataframe

,Name,Ticker,Current Price,HQM Score,Market Capitalization,24H Price Return,24H Return Percentile,7D Price Return,7D Return Percentile,14D Price Return,14D Return Percentile,30D Price Return,30D Return Percentile,200D Price Return,200D Return Percentile,1Y Price Return,1Y Return Percentile,Number of Tokens to Buy
0,Bitcoin,btc,39142,N/A,742988405806,-3040.696184,N/A,-11.001182,N/A,4.736174,N/A,-10.859757,N/A,-20.009486,N/A,-28.442205,N/A,N/A
1,Ethereum,eth,2596.04,N/A,311193529447,-159.554539,N/A,-12.097422,N/A,0.051634,N/A,-17.584787,N/A,-19.477657,N/A,38.875382,N/A,N/A
2,Tether,usdt,1.001,N/A,80181494275,0.000494,N/A,-0.028126,N/A,-0.101427,N/A,-0.035489,N/A,0.04619,N/A,-0.155798,N/A,N/A
3,BNB,bnb,377.37,N/A,63436097483,-17.696981,N/A,-7.797619,N/A,2.677987,N/A,-13.040244,N/A,-16.16903,N/A,28.042598,N/A,N/A
4,USD Coin,usdc,1.0,N/A,52310423078,0.000259,N/A,0.071215,N/A,-0.012933,N/A,0.123387,N/A,0.189143,N/A,-0.141977,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Gemini Dollar,gusd,1.0,N/A,286012046,-0.001811,N/A,-0.256572,N/A,-0.077035,N/A,-0.211286,N/A,0.829234,N/A,-0.245999,N/A,N/A
196,Ronin,ron,1.92,N/A,284211846,-0.100661,N/A,-15.069424,N/A,-13.026632,N/A,-35.011368,N/A,None,N/A,None,N/A,N/A
197,WazirX,wrx,0.61647,N/A,281461624,-0.005663,N/A,-3.702952,N/A,1.404305,N/A,-38.339782,N/A,-58.867196,N/A,61.613155,N/A,N/A
198,SafeMoon [OLD],safemoon,0.0,N/A,277800683,-0.0,N/A,-7.746769,N/A,-62.582417,N/A,-12.720391,N/A,-75.570325,N/A,None,N/A,N/A


# Calculating Momentum Percentiles

Next, we will calculate the momentum percentile scores for 24H, 7D, 14D, 30D, 200D and 1Y. We will first check if there are any missing values in the data obtained above using isnull() function.

In [10]:
hqm_dataframe[hqm_dataframe.isnull().any(axis=1)]
for column in ['30D Price Return', '200D Price Return','1Y Price Return']:
    hqm_dataframe[column].fillna(hqm_dataframe[column].mean(), inplace = True)

time_periods = [
                '24H',
                '7D',
                '14D',
                '30D',
                '200D',
                '1Y',
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
hqm_dataframe        

,Name,Ticker,Current Price,HQM Score,Market Capitalization,24H Price Return,24H Return Percentile,7D Price Return,7D Return Percentile,14D Price Return,14D Return Percentile,30D Price Return,30D Return Percentile,200D Price Return,200D Return Percentile,1Y Price Return,1Y Return Percentile,Number of Tokens to Buy
0,Bitcoin,btc,39142,N/A,742988405806,-3040.696184,0.005,-11.001182,0.465,4.736174,0.765,-10.859757,0.76,-20.009486,0.46,-28.442205,0.28,N/A
1,Ethereum,eth,2596.04,N/A,311193529447,-159.554539,0.03,-12.097422,0.365,0.051634,0.585,-17.584787,0.635,-19.477657,0.475,38.875382,0.535,N/A
2,Tether,usdt,1.001,N/A,80181494275,0.000494,0.895,-0.028126,0.85,-0.101427,0.53,-0.035489,0.87,0.046190,0.585,-0.155798,0.435,N/A
3,BNB,bnb,377.37,N/A,63436097483,-17.696981,0.065,-7.797619,0.625,2.677987,0.685,-13.040244,0.72,-16.169030,0.51,28.042598,0.52,N/A
4,USD Coin,usdc,1.0,N/A,52310423078,0.000259,0.885,0.071215,0.88,-0.012933,0.56,0.123387,0.875,0.189143,0.6,-0.141977,0.44,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Gemini Dollar,gusd,1.0,N/A,286012046,-0.001811,0.72,-0.256572,0.81,-0.077035,0.54,-0.211286,0.835,0.829234,0.63,-0.245999,0.42,N/A
196,Ronin,ron,1.92,N/A,284211846,-0.100661,0.435,-15.069424,0.16,-13.026632,0.105,-35.011368,0.2,29.753675,0.765,725.901375,0.87,N/A
197,WazirX,wrx,0.61647,N/A,281461624,-0.005663,0.665,-3.702952,0.74,1.404305,0.64,-38.339782,0.13,-58.867196,0.19,61.613155,0.575,N/A
198,SafeMoon [OLD],safemoon,0.0,N/A,277800683,-0.0,0.845,-7.746769,0.63,-62.582417,0.005,-12.720391,0.725,-75.570325,0.035,725.901375,0.87,N/A


# Calculating the HQM score

We will now calculate the 'HQM score' which is the high quality momentum score that we will use to filter cryptocurrencies for our investing strategy. 

HQM score will be the arithmetic mean of 7D Return Percentile, 14D Return Percentile, 30D Return Percentile, 200D Return Percentile, and 1Y Return Percentile.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module. 

In [11]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Name,Ticker,Current Price,HQM Score,Market Capitalization,24H Price Return,24H Return Percentile,7D Price Return,7D Return Percentile,14D Price Return,14D Return Percentile,30D Price Return,30D Return Percentile,200D Price Return,200D Return Percentile,1Y Price Return,1Y Return Percentile,Number of Tokens to Buy
0,Bitcoin,btc,39142,0.455833,742988405806,-3040.696184,0.005,-11.001182,0.465,4.736174,0.765,-10.859757,0.76,-20.009486,0.46,-28.442205,0.28,N/A
1,Ethereum,eth,2596.04,0.4375,311193529447,-159.554539,0.03,-12.097422,0.365,0.051634,0.585,-17.584787,0.635,-19.477657,0.475,38.875382,0.535,N/A
2,Tether,usdt,1.001,0.694167,80181494275,0.000494,0.895,-0.028126,0.85,-0.101427,0.53,-0.035489,0.87,0.046190,0.585,-0.155798,0.435,N/A
3,BNB,bnb,377.37,0.520833,63436097483,-17.696981,0.065,-7.797619,0.625,2.677987,0.685,-13.040244,0.72,-16.169030,0.51,28.042598,0.52,N/A
4,USD Coin,usdc,1.0,0.706667,52310423078,0.000259,0.885,0.071215,0.88,-0.012933,0.56,0.123387,0.875,0.189143,0.6,-0.141977,0.44,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Gemini Dollar,gusd,1.0,0.659167,286012046,-0.001811,0.72,-0.256572,0.81,-0.077035,0.54,-0.211286,0.835,0.829234,0.63,-0.245999,0.42,N/A
196,Ronin,ron,1.92,0.4225,284211846,-0.100661,0.435,-15.069424,0.16,-13.026632,0.105,-35.011368,0.2,29.753675,0.765,725.901375,0.87,N/A
197,WazirX,wrx,0.61647,0.49,281461624,-0.005663,0.665,-3.702952,0.74,1.404305,0.64,-38.339782,0.13,-58.867196,0.19,61.613155,0.575,N/A
198,SafeMoon [OLD],safemoon,0.0,0.518333,277800683,-0.0,0.845,-7.746769,0.63,-62.582417,0.005,-12.720391,0.725,-75.570325,0.035,725.901375,0.87,N/A


# Selecting the Top 20 Momentum Cryptocurrencies

We can select the top 20 momentum cryptocurrencies by sorting our DataFrame on the 'HQM Score' column and dropping all cryptocurrencies outside the top 20 rank.

In [12]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:20]
hqm_dataframe.reset_index(inplace = True)
hqm_dataframe

,index,Name,Ticker,Current Price,HQM Score,Market Capitalization,24H Price Return,24H Return Percentile,7D Price Return,7D Return Percentile,14D Price Return,14D Return Percentile,30D Price Return,30D Return Percentile,200D Price Return,200D Return Percentile,1Y Price Return,1Y Return Percentile,Number of Tokens to Buy
0,48,Waves,waves,26.86,0.885833,2837487575,3.15,1.0,48.884693,0.99,186.018056,0.995,140.054995,0.995,7.413504,0.66,149.992683,0.675,N/A
1,121,Xido Finance,xido,20.92,0.84,591261882,0.372933,0.985,274.629559,1.0,236.12484,1.0,191.380122,1.0,-59.041789,0.185,725.901375,0.87,N/A
2,192,Kyber Network Crystal,knc,2.84,0.8225,291661118,-0.137109,0.395,1.373092,0.9,63.85342,0.985,42.140861,0.97,42.424740,0.815,725.901375,0.87,N/A
3,122,Songbird,sgb,0.075832,0.81,590960207,0.001524,0.905,-2.372429,0.765,30.275021,0.93,-18.628427,0.625,29.753675,0.765,725.901375,0.87,N/A
4,6,Terra,luna,95.86,0.798333,34885934028,-2.255514,0.175,4.229007,0.94,60.320331,0.98,60.817626,0.98,247.418194,0.955,716.465399,0.76,N/A
5,187,FLEX Coin,flex,3.91,0.791667,307570609,0.000287,0.89,-0.997636,0.795,-2.500925,0.415,-10.425585,0.78,99.078521,0.89,1170.307055,0.98,N/A
6,27,LEO Token,leo,5.83,0.764167,5462238692,0.228183,0.975,-1.631172,0.785,-4.525494,0.3,20.985976,0.96,95.581746,0.88,192.219186,0.685,N/A
7,148,Golem,glm,0.471947,0.756667,473984270,-0.033577,0.54,5.576177,0.955,47.689304,0.97,12.400554,0.95,8.571266,0.665,2.849409,0.46,N/A
8,171,Hive,hive,0.946922,0.756667,353162921,0.047976,0.94,-5.959635,0.685,13.190098,0.865,-24.117801,0.49,87.321487,0.87,197.007763,0.69,N/A
9,49,Magic Internet Money,mim,0.996673,0.7525,2777408775,-0.000761,0.77,-0.046935,0.845,0.070504,0.59,-0.044986,0.86,0.000616,0.58,725.901375,0.87,N/A


# Calculating the Number of Tokens to Buy

Next we will calculate the number of tokens to buy using the portfolio_input function. We'll use the portfolio_input function that we created earlier to accept our portfolio size.

In [13]:
portfolio_input()

Enter the value of your portfolio:1000000


In [14]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Tokens to Buy'] = math.floor(position_size / hqm_dataframe['Current Price'][i])
hqm_dataframe

,index,Name,Ticker,Current Price,HQM Score,Market Capitalization,24H Price Return,24H Return Percentile,7D Price Return,7D Return Percentile,14D Price Return,14D Return Percentile,30D Price Return,30D Return Percentile,200D Price Return,200D Return Percentile,1Y Price Return,1Y Return Percentile,Number of Tokens to Buy
0,48,Waves,waves,26.86,0.885833,2837487575,3.15,1.0,48.884693,0.99,186.018056,0.995,140.054995,0.995,7.413504,0.66,149.992683,0.675,1861
1,121,Xido Finance,xido,20.92,0.84,591261882,0.372933,0.985,274.629559,1.0,236.12484,1.0,191.380122,1.0,-59.041789,0.185,725.901375,0.87,2390
2,192,Kyber Network Crystal,knc,2.84,0.8225,291661118,-0.137109,0.395,1.373092,0.9,63.85342,0.985,42.140861,0.97,42.424740,0.815,725.901375,0.87,17605
3,122,Songbird,sgb,0.075832,0.81,590960207,0.001524,0.905,-2.372429,0.765,30.275021,0.93,-18.628427,0.625,29.753675,0.765,725.901375,0.87,659352
4,6,Terra,luna,95.86,0.798333,34885934028,-2.255514,0.175,4.229007,0.94,60.320331,0.98,60.817626,0.98,247.418194,0.955,716.465399,0.76,521
5,187,FLEX Coin,flex,3.91,0.791667,307570609,0.000287,0.89,-0.997636,0.795,-2.500925,0.415,-10.425585,0.78,99.078521,0.89,1170.307055,0.98,12787
6,27,LEO Token,leo,5.83,0.764167,5462238692,0.228183,0.975,-1.631172,0.785,-4.525494,0.3,20.985976,0.96,95.581746,0.88,192.219186,0.685,8576
7,148,Golem,glm,0.471947,0.756667,473984270,-0.033577,0.54,5.576177,0.955,47.689304,0.97,12.400554,0.95,8.571266,0.665,2.849409,0.46,105944
8,171,Hive,hive,0.946922,0.756667,353162921,0.047976,0.94,-5.959635,0.685,13.190098,0.865,-24.117801,0.49,87.321487,0.87,197.007763,0.69,52802
9,49,Magic Internet Money,mim,0.996673,0.7525,2777408775,-0.000761,0.77,-0.046935,0.845,0.070504,0.59,-0.044986,0.86,0.000616,0.58,725.901375,0.87,50166


Finally we will output the above dataframe in an excel file. 

In [15]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [16]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [17]:
column_formats = { 
                    'A': ['index', integer_template],        
                    'B': ['Name', string_template],
                    'C': ['Ticker', string_template],
                    'D': ['Current Price', dollar_template],
                    'E': ['HQM Score', float_template],
                    'F': ['Market Capitalization', integer_template],
                    'G': ['24H Price Return', percent_template],
                    'H': ['24H Return Percentile', percent_template],
                    'I': ['7D Price Return', percent_template],
                    'J': ['7D Return Percentile', percent_template],
                    'K': ['14D Price Return', percent_template],
                    'L': ['14D Return Percentile', percent_template],
                    'M': ['30D Price Return', percent_template],
                    'N': ['30D Return Percentile', percent_template],
                    'O': ['200D Price Return', percent_template],
                    'P': ['200D Return Percentile', percent_template],
                    'Q': ['1Y Price Return', percent_template],
                    'R': ['1Y Return Percentile', percent_template],
                    'S': ['Number of Tokens to Buy', integer_template],
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [18]:
writer.save()